# Explore Co-clustering on Job Applications

### Import necessary packages:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
from sklearn.datasets import make_biclusters
from sklearn.datasets import make_checkerboard
from sklearn.datasets import samples_generator as sg
from sklearn.cluster import KMeans
from sklearn.cluster.bicluster import SpectralCoclustering
from random import shuffle

### Config constants:

In [ ]:
raw_data_path = './apps.tsv'

number_of_fold = 5 # for cross validation
one_time_run = True # turn it on if we don't want to do 5-fold cross validation which might be slow

# config # of cluster
min_cluster_num = 2
max_cluster_num = 105
cluster_num_step = 10

# flags to turn on/off algorithms
run_kmeans_flag = False
run_spectral_coclustering_flag = True
run_nmtf_flag = False

# enable visualizing clustering result on user-job matrix
enable_visualization = True

# threshold of popular jobs based # of applications per job,
# each of them will generate a user-job matrix with different densities
thresholds_popular_jobs = [100, 75] # 100 leads to more dense matrix with less entries, 75 leads to more sparse matrix with more entries

### Load dataset

In [ ]:
with open(raw_data_path, 'r') as fh:
    headers = fh.readline().strip().split('\t')
    content = fh.readlines()
raw_apps = [x.rstrip('\r\n').split('\t') for x in content]

job_apps_cnt = {}
for app in raw_apps:
    jobId, memberId = app[-1], app[0]
    if jobId in job_apps_cnt:
        job_apps_cnt[jobId] = job_apps_cnt[jobId] + 1
    else:
        job_apps_cnt[jobId] = 1

### Define method to pick applications for different thresholds of popular jobs and split into train and test sets for k-fold cross validation

In [ ]:
# prepross and split applications for k-fold cross validation
def create_train_test_set(threshold_popular_jobs, raw_apps, job_apps_cnt):
    popular_jobs = set()
    for jobId, cnt in job_apps_cnt.items():
        if cnt >= threshold_popular_jobs:
            popular_jobs.add(jobId)
        
    #print('# of popular jobs:', len(popular_jobs))

    unique_apps = []
    for app in raw_apps:
            jobId, memberId = app[-1], app[0]
            if jobId in popular_jobs:
                unique_apps.append((memberId, jobId))

    shuffle(unique_apps)
    
    splats = np.array_split(unique_apps, number_of_fold)

    train_set, test_set = {}, {}
    for i in range(number_of_fold):
        train_set[i] = []
        for j in range(number_of_fold):
            if i != j:
                train_set[i] = train_set[i] + splats[j].tolist()
        test_set[i] = splats[i].tolist()
    return train_set, test_set, popular_jobs
    

### Define helper functions to get users, jobs and applications for training and testing set

In [ ]:
def get_train_summary(train_apps, popular_jobs):
    train_user = set()
    train_job = set()
    train_rel = set()
    for app in train_apps:
        jobId, memberId = app[-1], app[0]
        if jobId in popular_jobs:
            train_user.add(memberId)
            train_job.add(jobId)
            train_rel.add((memberId, jobId))
    return train_user, train_job, train_rel


def get_test_summary(test_apps, train_user, train_job, train_rel):
    test_user = set()
    test_job = set()
    test_rel = set()
    for app in test_apps:
        jobId, memberId = app[-1], app[0]
        # we want to test clustered jobs and users in training set
        if jobId in train_job and memberId in train_user:
            test_user.add(memberId)
            test_job.add(jobId)
            # we only consider the applications from testing set which were not used for clustering during training process
            if (memberId, jobId) not in train_rel:
                test_rel.add((memberId, jobId))
    return test_user, test_job, test_rel

### Define some helper methods for prerocessing and building user-job matrix because in the raw data, user and job are respresented by ids so we need to map ids to index before creating user-job matrix.

In [ ]:
def mapUserAndJobToIndex(apps, popular_jobs):
    jobIdToIndex = {}
    memberIdToIndex = {}
    jidx = 0
    midx = 0
    popular_member_ids = set()
    for app in apps:
        jobId, memberId = app[-1], app[0]
        if jobId in popular_jobs:
            popular_member_ids.add(memberId)
            if jobId not in jobIdToIndex:
                jobIdToIndex[jobId] = jidx
                jidx = jidx + 1
            if memberId not in memberIdToIndex:
                memberIdToIndex[memberId] = midx
                midx = midx + 1
    return jobIdToIndex, memberIdToIndex, popular_member_ids
                
def buildUserJobMatrix(apps, user_job_mat, jobIdToIndex, memberIdToIndex):
    member_apps = {} # store jobs each member has applied
    for app in apps:
        jobId, memberId = app[-1], app[0]
        if jobId in jobIdToIndex:
            midx = memberIdToIndex[memberId]
            jidx = jobIdToIndex[jobId]
            user_job_mat[memberIdToIndex[memberId]][jobIdToIndex[jobId]] = 1
            if midx in member_apps:
                member_apps.get(midx).add(jidx)
            else:
                new_app_set = set()
                new_app_set.add(jidx)
                member_apps[midx] = new_app_set
    print('user-job matrix size:', user_job_mat.shape)
    return user_job_mat, member_apps
    

### Define validation method for K-means and NMTF(because this co-clustering method doesn't tell which row cluster corresponds to which column cluster as it requires to set # of row and column clusters separately) to generate metrics.

#### True Positive: a user applies to a job in testing set and they are in the same trained cluster.
#### True Negative: a user doesn't apply to a job in testing set and they are in the different trained clusters.
#### False Positive: a user doesn't apply to a job in testing set but they are in the same trained cluster.
#### False Negative: a user applies to a job in testing set but they are in the different trained clusters.

In [ ]:
def validateKmeansOrNMTF(test_user, test_job, test_rel, user_cluster_map, clusters_index_map, member_apps, memberIdToIndex, jobIdToIndex):
    cluster_jobs = {}
    # cluster the jobs that are applied by the users in the same user cluster
    for label in clusters_index_map:
        m_indices = clusters_index_map.get(label)
        jobs = set()
        for midx in m_indices:
            # get jobs for each member in cluster
            jobs = jobs.union(member_apps[midx])
        cluster_jobs[label] = jobs 
        
    TP, FN = 0, 0
    for (memberId, jobId) in test_rel:
        midx = memberIdToIndex[memberId]
        jidx = jobIdToIndex[jobId]
        if midx in user_cluster_map:
            label = user_cluster_map[midx]
            if jidx in cluster_jobs[label]:
                TP = TP + 1
            else:
                FN = FN + 1
                
    TN, FP = 0, 0
    for memberId in test_user:
        for jobId in test_job:
            if (memberId, jobId) not in test_rel:
                midx = memberIdToIndex[memberId]
                jidx = jobIdToIndex[jobId]
                if midx in user_cluster_map:
                    label = user_cluster_map[midx]
                    if jidx in cluster_jobs[label]:
                        FP = FP + 1
                    else:
                        TN = TN + 1
                
    if TP == 0 and FN == 0:
        recall = 0
    else:
        recall = TP * 1.0 / (TP +FN)
    
    pre = TP * 1.0 / (TP + FP)
    recall = TP * 1.0 / (TP +FN)
    F1 = 2.0 * pre * recall /(pre +recall)
    accuracy = (TP + TN) * 1.0 /(TP + TN + FP + FN)
    
    print('Validation precision, recall, F1, accuracy', pre, recall, F1, accuracy)
   
    return recall, F1, accuracy

### Define method for K-means including training and testing

In [ ]:
def run_kmeans(train_set, test_set, popular_jobs):
    print('Start running K-means')
    recall_map = {}
    f1_map = {}
    accuracy_map = {}
    for clusters_cnt in xrange(min_cluster_num, max_cluster_num, cluster_num_step):
        print('Clustering with cluster count:', clusters_cnt)
        
        experiment_cnt = number_of_fold if not one_time_run else 1
            
        recalls = [0] * experiment_cnt
        f1s = [0] * experiment_cnt
        accuracies = [0] * experiment_cnt

        for fold in range(experiment_cnt):
            apps = train_set[fold]
            jobIdToIndex, memberIdToIndex, popular_member_ids = mapUserAndJobToIndex(apps, popular_jobs)
            user_job_mat = np.zeros((len(popular_member_ids), len(jobIdToIndex)))
            user_job_mat, member_apps = buildUserJobMatrix(apps, user_job_mat, jobIdToIndex, memberIdToIndex)

            est = KMeans(n_clusters=clusters_cnt)
            est.fit(user_job_mat)
            labels = est.labels_

            clusters_index_map = {} # cluster label -> user index
            user_cluster_map = {} # user index -> cluster label
            for i, label in enumerate(labels):
                user_cluster_map[i] = label
                if label in clusters_index_map:
                    clusters_index_map[label].append(i)
                else:
                    clusters_index_map[label] = [i]

            if enable_visualization:
                # just for coloring
                for label in range(clusters_cnt):
                    for i in clusters_index_map[label]:
                        user_job_mat[i] = user_job_mat[i] * (label + 1)

                # group users in same cluster together
                sorted_user_job_mat = user_job_mat[np.argsort(labels)]

                plt.figure(figsize = (5, 5))
                plt.ylabel('Users')
                plt.xlabel('Jobs')
                cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["white", "red","violet","blue"])
                plt.imshow(sorted_user_job_mat, cmap=cmap, aspect=user_job_mat.shape[1] * 1.0 / user_job_mat.shape[0])
                plt.show()

            train_user, train_job, train_rel = get_train_summary(apps, popular_jobs)
            test_apps = test_set[fold]
            test_user, test_job, test_rel = get_test_summary(test_apps, train_user, train_job, train_rel)
            recalls[fold], f1s[fold], accuracies[fold] = validateKmeansOrNMTF(test_user, test_job, test_rel, user_cluster_map, clusters_index_map, member_apps, memberIdToIndex, jobIdToIndex)

        recall_map[clusters_cnt] = np.mean(recalls)
        f1_map[clusters_cnt] = np.mean(f1s)
        accuracy_map[clusters_cnt] = np.mean(accuracies)
    print('Average recall, f1 and accuracy for each number of clusters:', recall_map, f1_map, accuracy_map)

### Implement nmtf and define method for nmtf (Orthogonal nonnegative matrix t-factorizations for clustering) including training and testing

In [ ]:
def nmtf(X, k, l, num_iter=50, debug_mode=True):
    print('Start running iterations for nmtf')
    m, n = X.shape
    est = KMeans(n_clusters=k)
    est.fit(X)
    V = est.cluster_centers_.T
    #print('inital V ', V)
    V = V
    est = KMeans(n_clusters=l)
    est.fit(X.T)
    U = est.cluster_centers_.T
    #print('inital U ', U)
    S = U.T.dot(X).dot(V)

    error_best = np.inf
    error = error_best

    for i in xrange(num_iter):
        # solve subproblem to update V
        V = V * (X.T.dot(U).dot(S) / V.dot(V.T).dot(X.T).dot(U).dot(S))

        # solve subproblem to update U
        test1 = U.dot(U.T).dot(X).dot(V).dot(S.T)
        if np.isnan(test1).any():
            print('solve U', U, S, V, X)
            break
        U = U * (X.dot(V).dot(S.T) / test1)

        # solve subproblem to update S
        S = S * (U.T.dot(X).dot(V) / U.T.dot(U).dot(S).dot(V.T).dot(V))
        
        error_ant = error
        error = np.sum((X - U.dot(S).dot(V.T)) ** 2)

        if error < error_best:
            U_best = U
            S_best = S
            V_best = V
            error_best = error

        if debug_mode:
            print('Iteration No. and current error:', i, error)
        if np.abs(error - error_ant) <= 0.001:
            break

    rows_ind = np.argmax(U_best, axis=1)
    cols_ind = np.argmax(V_best, axis=1)

    return U_best, S_best, V_best.T, rows_ind, cols_ind, error_best


def run_nmtf(train_set, test_set, popular_jobs):
    print('Start running NMTF for co-clustering')
    recall_map = {}
    f1_map = {}
    accuracy_map = {}
    for clusters_cnt in xrange(min_cluster_num, max_cluster_num, cluster_num_step):
        print('Clustering with cluster count:', clusters_cnt)
        
        experiment_cnt = number_of_fold if not one_time_run else 1
            
        recalls = [0] * experiment_cnt
        f1s = [0] * experiment_cnt
        accuracies = [0] * experiment_cnt

        for fold in range(experiment_cnt):
            apps = train_set[fold]
            jobIdToIndex, memberIdToIndex, popular_member_ids = mapUserAndJobToIndex(apps, popular_jobs)
            
            user_job_mat = np.zeros((len(popular_member_ids), len(jobIdToIndex)))
             # initialize to 0.01 to avoid devide by zero issue when running NMTF, it won't affect the result
            for row in range(len(popular_member_ids)):
                for col in range(len(jobIdToIndex)):
                    user_job_mat[row][col] = 0.01
            
            user_job_mat, member_apps = buildUserJobMatrix(apps, user_job_mat, jobIdToIndex, memberIdToIndex)

            # normally row cluster number equals to column cluster number
            U_best, S_best, V_best, rows_ind, cols_ind, error_best = nmtf(user_job_mat, clusters_cnt, clusters_cnt)
            labels = rows_ind
            clusters_index_map = {}
            user_cluster_map = {}
            for i, label in enumerate(labels):
                user_cluster_map[i] = label
                if label in clusters_index_map:
                    clusters_index_map[label].append(i)
                else:
                    clusters_index_map[label] = [i]

            if enable_visualization:
                fit_data = user_job_mat[np.argsort(rows_ind)]
                fit_data = fit_data[:, np.argsort(cols_ind)]
                
                # find cluster separation coordinates on x and y axis
                rows_sep = set()
                cols_sep = set()
                sorted_rows_ind = np.sort(rows_ind) 
                sorted_cols_ind = np.sort(cols_ind)
                for i in range(rows_ind.shape[0]):
                    if i != 0 and sorted_rows_ind[i] != sorted_rows_ind[i-1]:
                        rows_sep.add(i)
                for i in range(cols_ind.shape[0]):
                    if i != 0 and sorted_cols_ind[i] != sorted_cols_ind[i-1]:
                        cols_sep.add(i)

                cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["white", "blue"])

                plt.figure(figsize = (8, 8))
                plt.ylabel('Users')
                plt.xlabel('Jobs')
                plt.imshow(fit_data, cmap=cmap, aspect=user_job_mat.shape[1] * 1.0 / user_job_mat.shape[0])
                
                # draw cluster separation lines
                for row_sep in rows_sep:
                    plt.axhline(y = row_sep, linewidth=1, color='r')
                for col_sep in cols_sep:
                    plt.axvline(x = col_sep, linewidth=1, color='r')
                plt.show()

            train_user, train_job, train_rel = get_train_summary(apps, popular_jobs)
            test_apps = test_set[fold]
            test_user, test_job, test_rel = get_test_summary(test_apps, train_user, train_job, train_rel)
            recalls[fold], f1s[fold], accuracies[fold] = validateKmeansOrNMTF(test_user, test_job, test_rel, user_cluster_map, clusters_index_map, member_apps, memberIdToIndex, jobIdToIndex)


        recall_map[clusters_cnt] = np.mean(recalls)
        f1_map[clusters_cnt] = np.mean(f1s)
        accuracy_map[clusters_cnt] = np.mean(accuracies)
    print('Average recall, f1 and accuracy for each number of clusters:', recall_map, f1_map, accuracy_map)

### Define validation method for spectral co-clustering (Co-clustering documents and words using Bipartite Spectral Graph Partitioning)

In [ ]:
# validate spectral co-clustering
def validateSpectralCoClustering(test_user, test_job, test_rel, user_cluster_map, job_cluster_map, memberIdToIndex, jobIdToIndex):
    TP, FN = 0, 0
    cluster_dist = {}
    true_dist = {}
    for (memberId, jobId) in test_rel:
        midx = memberIdToIndex[memberId]
        jidx = jobIdToIndex[jobId] 
        if midx in user_cluster_map and jidx in job_cluster_map:
            true_cluster = user_cluster_map[midx]
            if true_cluster in true_dist:
                true_dist[true_cluster] = true_dist[true_cluster] + 1
            else:
                true_dist[true_cluster] = 1
                
            if user_cluster_map[midx] == job_cluster_map[jidx]:
                TP = TP + 1
                cluster = user_cluster_map[midx]
                if cluster in cluster_dist:
                    cluster_dist[cluster] = cluster_dist[cluster] + 1
                else:
                    cluster_dist[cluster] = 1
            else:
                FN = FN + 1
    #print('test data cluster and true dist:', cluster_dist, true_dist)
    TN, FP = 0, 0
    for memberId in test_user:
        for jobId in test_job:
            if (memberId, jobId) not in test_rel:
                midx = memberIdToIndex[memberId]
                jidx = jobIdToIndex[jobId]
                if midx in user_cluster_map and jidx in job_cluster_map:
                    if user_cluster_map[midx] == job_cluster_map[jidx]:
                        FP = FP + 1
                    else:
                        TN = TN + 1
                        
    print('TP:', TP)
    pre = TP * 1.0 / (TP + FP)
    recall = TP * 1.0 / (TP +FN)
    F1 = 2.0 * pre * recall /(pre +recall)
    accuracy = (TP + TN) * 1.0 /(TP + TN + FP + FN)
    print('test data precision, recall, F1, accuracy', pre, recall, F1, accuracy)
  
    return recall, F1, accuracy

### Define method to compute the average silhouette score of clusters

In [ ]:
def getSilhouetteScore(member_apps, model, inverted_user_cluster_map, inverted_job_cluster_map):
    member_cnt = len(member_apps)
    score_dis_in_cluster = np.zeros(member_cnt) # disimilarity for each user within a cluster
    for i in member_apps:
        label = model.row_labels_[i]
        if label in inverted_job_cluster_map:
            jobs_in_cluster = inverted_job_cluster_map.get(label)
            app_in_cluster = len(jobs_in_cluster.intersection(member_apps.get(i)))
        else:
            app_in_cluster = 0
        total_app = len(member_apps.get(i))
        score_dis_in_cluster[i] = 1 -  app_in_cluster /total_app

    score_dis_out_cluster = np.zeros(member_cnt) # disimilarity for each user with different clusters
    for i in member_apps:
        total_app = len(member_apps.get(i))
        b = -1 #disimilarity with one cluster
        for label in inverted_job_cluster_map:
            if label != model.row_labels_[i]:
                jobs_in_cluster = inverted_job_cluster_map.get(label)
                app_in_cluster = len(jobs_in_cluster.intersection(member_apps.get(i)))
                b_per_label = 1 -  app_in_cluster /total_app
                if b == -1 or b_per_label < b:
                    b = b_per_label
        score_dis_out_cluster[i] = b # minimum dismilarity with different clusters

    s_score = {}
    cluster_score = {} # cluster silhouette score equals to the average silhouette score of users in the cluster
    total_s_score = 0 # total cluster silhouette scores
    for label in inverted_user_cluster_map:
        members_in_cluster = inverted_user_cluster_map.get(label)
        score = 0
        for i in  members_in_cluster:
            b = score_dis_out_cluster[i]
            a = score_dis_in_cluster[i]
            s_score[i] = (b - a) / max(a, b)
            score = score + s_score[i]
        cluster_score[label] = score / len(members_in_cluster)
        total_s_score = total_s_score + cluster_score[label]
    return total_s_score/len(cluster_score)

### Define methods to train and test using Spectral Co-clustering

In [ ]:
def run_spectral_coclustering(train_set, test_set, popular_jobs):
    print('Start running Spectral Co-clustering')
    recall_map = {}
    f1_map = {}
    accuracy_map = {}
    s_score_map = {} # silhouette score
    for clusters_cnt in xrange(min_cluster_num, max_cluster_num, cluster_num_step):
        print('Clustering with cluster count:', clusters_cnt)
        
        experiment_cnt = number_of_fold if not one_time_run else 1
            
        recalls = [0] * experiment_cnt
        f1s = [0] * experiment_cnt
        accuracies = [0] * experiment_cnt
        s_scores = [0] * experiment_cnt
        for fold in range(experiment_cnt):
            apps = train_set[fold]
            jobIdToIndex, memberIdToIndex, popular_member_ids = mapUserAndJobToIndex(apps, popular_jobs)
            user_job_mat = np.zeros((len(popular_member_ids), len(jobIdToIndex)))
            user_job_mat, member_apps = buildUserJobMatrix(apps, user_job_mat, jobIdToIndex, memberIdToIndex)

            model = SpectralCoclustering(n_clusters=clusters_cnt, random_state=0)
            model.fit(user_job_mat)

            labels = model.row_labels_
            user_cluster_map = {}
            inverted_user_cluster_map = {}
            for i, label in enumerate(labels):
                user_cluster_map[i] = label
                if label in inverted_user_cluster_map:
                    inverted_user_cluster_map[label].add(i)
                else:
                    new_set = set()
                    new_set.add(i)
                    inverted_user_cluster_map[label] = new_set

            labels = model.column_labels_
            job_cluster_map = {}
            inverted_job_cluster_map = {}
            for i, label in enumerate(labels):
                job_cluster_map[i] = label
                if label in inverted_job_cluster_map:
                    inverted_job_cluster_map[label].add(i)
                else:
                    new_set = set()
                    new_set.add(i)
                    inverted_job_cluster_map[label] = new_set
                    
            if enable_visualization:
                colored_fit_data = np.copy(user_job_mat)
                # coloring original user-job matrix
                for label in range(clusters_cnt):
                    if label in inverted_user_cluster_map and label in inverted_job_cluster_map:
                        for i in inverted_user_cluster_map[label]:
                            for j in inverted_job_cluster_map[label]:
                               colored_fit_data[i][j] = colored_fit_data[i][j] * (label + 1)

                colored_fit_data = colored_fit_data[np.argsort(model.row_labels_)]
                colored_fit_data = colored_fit_data[:, np.argsort(model.column_labels_)]

                cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["white", "red","violet","blue"])
                plt.figure(figsize = (7, 7))
                plt.ylabel('Users')
                plt.xlabel('Jobs')
                plt.imshow(colored_fit_data, cmap=cmap, aspect=user_job_mat.shape[1] * 1.0 / user_job_mat.shape[0])
                plt.show()
        
            
            train_user, train_job, train_rel = get_train_summary(apps, popular_jobs)
            test_apps = test_set[fold]
            test_user, test_job, test_rel = get_test_summary(test_apps, train_user, train_job, train_rel)
            recalls[fold], f1s[fold], accuracies[fold] = validateSpectralCoClustering(test_user, test_job, test_rel, user_cluster_map, job_cluster_map, memberIdToIndex, jobIdToIndex)
            s_scores[fold] = getSilhouetteScore(member_apps, model, inverted_user_cluster_map, inverted_job_cluster_map)

        recall_map[clusters_cnt] = np.mean(recalls)
        f1_map[clusters_cnt] = np.mean(f1s)
        accuracy_map[clusters_cnt] = np.mean(accuracies)
        s_score_map[clusters_cnt] = np.mean(s_scores)
    print('Average recall, f1, accuracy and silhouette score for each number of clusters:', recall_map, f1_map, accuracy_map, s_score_map)

### Run k-means, nmtf and spectral co-clustering on two dataset with different densities

In [ ]:
for threshold_popular_jobs in thresholds_popular_jobs:
    train_sets, test_sets, popular_jobs = create_train_test_set(threshold_popular_jobs, raw_apps, job_apps_cnt)
    
    if run_kmeans_flag:
        run_kmeans(train_sets, test_sets, popular_jobs)
    if run_spectral_coclustering_flag:
        run_spectral_coclustering(train_sets, test_sets, popular_jobs)
    if run_nmtf_flag:
        run_nmtf(train_sets, test_sets, popular_jobs)
    
    